# 1.6 - Getting Variance of Interest Rates for Meeting & non-Meeting Days

This script expands on 1.2 by getting ED6, ED12 and ED24 movements, as well as movements in implied expectations of the FFR between the upcoming meeting $(t)$ and the meeting after $(t+1)$, and between the meeting after $(t+1)$ and the meeting after that $(t+2)$, for all days between 13 December 1993 and 31 December 2013 for which...

- Dates for meetings $t$ and $t+1$ are contained in `FOMC_dates`
- The last price on the previous day is also available

This delivers data for 3,749 non-meeting days and for the original 158 meeting days from script 1.2.

Variance for the movement of each interest rate for each subset of the total 3,907 days is then taken in the last two code blocks. 

### Preamble

This script makes use of...

- Pandas
- NumPy
- Regular Expressions (`re`)
- `calendar`

In [41]:
import pandas as pd
import numpy as np
import re
from calendar import monthrange
from pandas.tseries.offsets import DateOffset

### Import Meeting Dates

In [42]:
FOMC_dates = []

with open('dates.csv','r') as FOMC_dates_file:
    
    for line in FOMC_dates_file: # The "for" loop is somewhat redundant - the file has only one line.
        
        raw_FOMC_dates = line.split(',')

for date in raw_FOMC_dates:
    
    unix_date = int(re.search("[0-9]+", date).group(0)) # isolates integer from string

    FOMC_dates.append(pd.Timestamp((unix_date + 43200) - ((unix_date + 43200) % 86400), unit = 's')) # Prevents errors rounding down to 23:00 on previous day

FOMC_dates.append(pd.Timestamp((1390996800) - (1390996800 %86400), unit = 's'))

### Interest Rate Dataframe

This block compiles a dataframe of daily prices for FF1, FF2, FF3, FF4, FF5, ED6, ED12 and ED24 from December 13, 1993 to December 31 1994.

Unlike script 1.2, FF5 is included here in order to accomodate for days on which the appropriate Futures contract from which to imply shifts in expectations of the FFR between the meeting *after* the upcoming meeting and the meeting after that is 5 months out. 

In [43]:
ir_df = pd.read_excel('FF1.xlsx').set_index('Date').drop(['Open','High','Low','Open Interest','SMAVG (15)'], axis = 1)

# Bloomberg's FF1 series has a different data structure to all others, so is treated as above.

ir_df = ir_df.rename({'Close':'FF1'}, axis = 1)

def merge_df(df_main, x): # This function gets all data from FF1, FF2, FF3, FF4, ED6, ED12 and ED24 into the same dataframe
    
    print(x + ' added')
    
    df_r = pd.read_excel(x + '.xlsx') # for example, 'FF3.xlsx'
    
    df_r = df_r.set_index('Date').drop(['Open Interest','SMAVG (15)'], axis = 1).rename(columns = {'Last Price':x})
    
    return pd.concat([df_main, df_r], axis=1, join = 'inner')

for ir in ['FF2','FF3','FF4','FF5','ED6','ED12','ED24']:
    
    ir_df = merge_df(ir_df, ir)

FF2 added
FF3 added
FF4 added
FF5 added
ED6 added
ED12 added
ED24 added


### Getting a Price *Movements* Dataframe; Refining the Dataframe

This code block creates a price movement dataframe, removing all days on which analysis cannot feasibly be conducted without error.

In [44]:
for date in ir_df.index:
    
    for ir in ir_df.columns:
        
        if date.day == 1 and ir in ['FF1','FF2','FF3','FF4']: # Handles contract switch on first day of month
            
            ir_df.loc[date,ir] = (100 - ir_df.loc[date,ir]) - (100 - ir_df.shift(-1).shift(-1, axis = 1).loc[date, ir])
        
        else:
            
            ir_df.loc[date,ir] = (100 - ir_df.loc[date,ir]) - (100 - ir_df.shift(-1).loc[date, ir])

ir_df = ir_df.drop(index = ir_df.index[[date > FOMC_dates[-2] for date in ir_df.index]]) # Removes days for shock to FFR expectations two meetings forward cannot be calculated 
ir_df = ir_df.drop(index = ir_df.index[[date < FOMC_dates[0] for date in ir_df.index]]) # Removes days prior to first meeting date.

fixed_index = ir_df.index # Fixing the ir_df index allows for rows to be dropped by index from the ir_df dataframe without rearranging the indices during the for loop. 

dbe = [(date + DateOffset(days = -1) in ir_df.index) for date in ir_df.index] # "Day Before Exists": For each day, checks the previous day also has a price

for i in range(len(dbe)):
    
    if dbe[i] == False:
        
        ir_df = ir_df.drop(index = [fixed_index[i]]) # Drops rows for which the pre-calculated movement was based on a price other than that of the day before 

### Calculating Implied Movements in Expectations of the FFR

Borrowing for script 1.2, implied movements are calculated as below.

Let...

- $m_t$ denote the number of days in the month which the upcoming meeting, $t$, occurs.
- $m_{t+1}$ denote the number of days in the month when the next meeting, $t+1$ occurs
- $d_t$ denote the day of the month on which meeting $t$ occurs
- $d_{t+1}$ denote the day of month on which meeting $t+1$ occurs
- $s_j$ denote the current day shock to the FFF corresponding to the month in which meeting $t$ occurs.
- $s_{j+1}$ denote the current day shock to the FFF corresponding to the month after that in which meeting $t$ occurs.
- $s_k$ denote the current day shock to the FFF corresponding to the month in which meeting $t+1$ occurs.
- $s_{k+1}$ denote the current day shock to the FFF corresponding to the month after that in which meeting $t+1$ occurs.

If meeting $t$ occurs outside the last week of its month, the shock to expectations of the FFR between meeting $t$ and $t + 1$ is given by...

$\phi_{t,t+1} = \frac{m_t}{m_t-d_t}s_j$

If it occurs *during* the last week of its month, the shock to expectations of the FFR between meeting $t$ and $t + 1$ is given by...

$\phi_{t,t+1} = s_{j+1}$

If meeting $t+1$ occurs outside the last week of its month, the shock to expectations of the FFR between meeting $t+1$ and $t + 2$ is given by...

$\phi_{t+1,t+2} = \frac{m_{t+1}}{m_{t+1}-d_{t+1}}\left(s_k - \frac{d_{t+1}}{m_{t+1}}\phi_{t,t+1}\right)$

If it occurs *during* the last week of its month, the shock to expectations of the FFR between meeting $t$ and $t + 1$ is given by...

$\phi_{t+1,t+2} = s_{k+1}$

In [45]:
nm = [] # Date of upcoming meeting

am = [] # Date of meeting after

months_ahead_nm = [] # Number of months until upcoming meeeting (would calculate Mar 1 as 2 months from Jan 31)

months_ahead_am = []

day_of_nm = [] # Day that upcoming meeting occurs in its month

day_of_am = []

days_in_nm = [] # Days in month of upcoming meeting

days_in_am = []

last_7_nm = [] # Is upcoming meeting in last week of its month (0 = False, 1 = True) 

last_7_am = []

nm_contract = [] # From which contract are expectations of the FFR from the upcoming meeting to the one after implied?

am_contract = []

nm_coefs = [] # Coefficient for implying next shock from single contract movement

am_coefs_1 = [] # Coefficient either m_{t+1}/(m_{t+1} - d_{t+1}) or 1, as above

am_coefs_2 = [] # Coefficient either d_{t+1}/m_{t+1} or 0, as above.

for date in ir_df.index:
    
    i = len(FOMC_dates) - sum([meeting >= date for meeting in FOMC_dates]) # Gets index of upcoming meeting in FOMC_dates
    
    next_meet, after_meet = FOMC_dates[i:i+2] # Gets dates into variables
    
    nm.append(next_meet)
    
    am.append(after_meet)
    
    m_ahead_nm = (next_meet.month - date.month) % 12 # timestamp.month returns an integer. Note that, for example, (2 - 12) % 12 = 2
    
    m_ahead_am = (after_meet.month - date.month) % 12
    
    months_ahead_nm.append(m_ahead_nm)
    
    months_ahead_am.append(m_ahead_am)
    
    do_nm = next_meet.day  # timestamp.day returns an integer. 
    
    day_of_nm.append(do_nm)
    
    do_am = after_meet.day
    
    day_of_am.append(do_am)
    
    di_nm = monthrange(next_meet.year,next_meet.month)[1] # monthrange(yyyy,m)[1] returns the number of days in the argued month
    
    days_in_nm.append(di_nm)
    
    di_am = monthrange(after_meet.year,after_meet.month)[1]
    
    days_in_am.append(di_am)
    
    if di_nm - do_nm <= 6: # i.e. if upcoming meeting in last week of its month
        
        l7_nm = 1
        
        last_7_nm.append(l7_nm)
        
        nm_coef = 1
        
        nm_coefs.append(nm_coef)
    
    else:
        
        l7_nm = 0
        
        last_7_nm.append(l7_nm)
    
        nm_coef = di_nm/(di_nm - do_nm)
        
        nm_coefs.append(nm_coef)
    
    if di_am - do_am <= 6: # i.e. if meeting after in last week of its month
        
        l7_am = 1
        
        last_7_am.append(l7_am)
        
        am_coef_1 = 1
        
        am_coefs_1.append(am_coef_1)
        
        am_coef_2 = 0
        
        am_coefs_2.append(am_coef_2)
    
    else:
        
        l7_am = 0
        
        last_7_am.append(l7_am)
    
        am_coef_1 = di_am/(di_am - do_am)
        
        am_coefs_1.append(am_coef_1)
        
        am_coef_2 = do_am/di_am
        
        am_coefs_2.append(am_coef_2)
    
    nm_con = 'FF' + str(l7_nm + m_ahead_nm + 1)  # Gets contract movement from which upcoming meeting shock implied
    
    nm_contract.append(nm_con)
    
    am_con = 'FF' + str(l7_am + m_ahead_am + 1)
    
    am_contract.append(am_con)
    
ir_df['nm'] = nm

ir_df['nm_contract'] = nm_contract

ir_df['nm_coef'] = nm_coefs

ir_df['am'] = am

ir_df['am_contract'] = am_contract

ir_df['am_coef_1'] = am_coefs_1

ir_df['am_coef_2'] = am_coefs_2

FFR0to1 = []

FFR1to2 = []

for date in ir_df.index: # This loop carries out the calculations given in the description above.
    
    mov1 = ir_df.loc[date,ir_df.loc[date,'nm_contract']]
    
    adj_mov1 = mov1*ir_df.loc[date,'nm_coef']
    
    FFR0to1.append(adj_mov1)
    
    mov2 = ir_df.loc[date,ir_df.loc[date,'am_contract']]
    
    adj_mov2 = ir_df.loc[date,'am_coef_1']*(mov2 - ir_df.loc[date,'am_coef_2']*adj_mov1)
    
    FFR1to2.append(adj_mov2)

ir_df['FFR0to1'] = FFR0to1

ir_df['FFR1to2'] = FFR1to2

ir_df = ir_df.drop(columns = ['FF1','FF2','FF3','FF4','FF5','nm','nm_contract','nm_coef','am','am_contract','am_coef_1','am_coef_2',])

### Get Non-meeting Day and Meeting Day Dataframes

In [46]:
ir_df_meetings = ir_df.loc[[(date in FOMC_dates) for date in ir_df.index]].dropna() # Gets meeting days for which values exist into one dataframe

ir_df_nonmeetings = ir_df.loc[[(date not in FOMC_dates) for date in ir_df.index]].dropna() # Gets non-meeting days for which values exist into one dataframe

### Data for Table 2

In [47]:
ir_df_nonmeetings.var() # Variances for columns of non-meeting-day dataframe

ED6        0.006285
ED12       0.005730
ED24       0.005616
FFR0to1    0.001602
FFR1to2    0.001384
dtype: float64

In [48]:
ir_df_meetings.var() # Variances for columns of meeting-day dataframe

ED6        0.014328
ED12       0.010630
ED24       0.007095
FFR0to1    0.002023
FFR1to2    0.002345
dtype: float64

### Data for Table 3

In [49]:
ir_df_nonmeetings.corr()

,ED6,ED12,ED24,FFR0to1,FFR1to2
ED6,1.000000,0.880319,0.708035,0.346181,0.478367
ED12,0.880319,1.000000,0.909903,0.280082,0.401335
ED24,0.708035,0.909903,1.000000,0.208093,0.312471
FFR0to1,0.346181,0.280082,0.208093,1.000000,0.369030
FFR1to2,0.478367,0.401335,0.312471,0.369030,1.000000
